In [1]:
import os
import sys
sys.path.append('..')
from backtest.operators import * 
from data_center.tej_handler import TEJHandler
from monitor.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import tejapi

import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
# TEJHandler()

In [2]:
position_list = ['2330', '1101']
p_open = TWMarketMonitor().get_position_open(position_list)
p_open

symbol,2330,1101
datetime,,
2024-04-02,NaN,NaN
2024-04-03,783.0,32.8


### Get Weighting

In [2]:
collection = "monthly_revenue"
monthly_rev_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [3]:
monthly_rev_pmart['revenue_yoy'] = monthly_rev_pmart['單月營收(千元)'] / monthly_rev_pmart['去年單月營收(千元)'] -1
example_factor = ts_rank(monthly_rev_pmart['revenue_yoy'],252)

In [4]:
def get_weighting(factor):
    demean = factor.sub(factor.mean(axis=1), axis=0)
    weighting = demean.div(demean.abs().sum(axis=1), axis=0)
    return weighting

### 假設出訊號後隔天收盤進場

In [5]:
example_signal = get_weighting(example_factor)
example_weighting = example_signal.shift(2) #隔天收盤進場，後天才有部位

### Get returns

In [6]:
collection = 'securities_returns'
returns_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [7]:
collection = 'securities_trading_data'
ohlcv_pmart = TWMarketMonitor().directly_get_pmart(collection)

In [65]:
ohlcv_pmart['開盤價']

symbol,0050,0051,0052,0053,0055,0056,0057,0061,006201,006203,...,9946,9949,9950,9951,9955,9957,9958,9960,9962,T1012Y
datetime,,,,,,,,,,,,,,,,,,,,,
2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.15,18.75,10.90,67.0,21.15,NaN,89.5,25.40,17.15,NaN
2022-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.20,19.10,10.50,67.1,20.85,NaN,88.4,26.20,17.35,NaN
2022-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.20,18.80,10.85,66.7,20.65,NaN,86.2,26.30,17.20,NaN
2022-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.25,18.85,10.85,66.1,20.40,NaN,88.0,26.45,17.10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-28,157.30,78.35,158.80,85.35,25.39,39.55,115.80,16.82,21.95,74.50,...,25.50,18.90,17.15,73.4,23.55,6.46,224.0,27.00,17.90,NaN
2024-03-29,156.90,79.00,158.45,85.30,25.18,39.75,116.85,16.92,21.96,75.25,...,25.70,18.70,16.90,72.7,25.00,6.44,223.0,27.00,17.90,NaN
2024-04-01,158.20,78.70,160.00,85.95,25.38,39.89,117.85,16.90,22.25,75.90,...,25.50,18.60,17.00,72.4,25.15,6.44,220.0,27.10,18.10,NaN


In [49]:
example_returns = returns_pmart['日報酬率(%)']*0.01

### 將weighting 與 returns結合獲得績效

In [55]:
stocks = 20
top = example_weighting.apply(lambda row: row.nlargest(stocks), axis=1).fillna(0)
example_equalweight = top.applymap(lambda x: 1/stocks if x != 0 else 0)
recent_w_weighting = example_equalweight#[-5:].dropna(axis =1, how = 'all')
recent_w_weighting = recent_w_weighting.loc[:, (recent_w_weighting != 0.0).any(axis=0)]
recent_w_weighting

/var/folders/fx/jn0gghjd4cg68lr15nlpsbqr0000gn/T/ipykernel_9314/681453456.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  example_equalweight = top.applymap(lambda x: 1/stocks if x != 0 else 0)


symbol,000116,000930,000960,0009A0,1101,1102,1104,1107,1108,1109,...,3228,3229,3231,3259,3373,4542,4905,5355,5478,5516
datetime,,,,,,,,,,,,,,,,,,,,,
1990-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1990-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-03-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
recent_w_returns = example_returns * recent_w_weighting
recent_w_profit = recent_w_returns.dropna(axis =1, how='all').dropna(axis=0, how='all')
recent_w_profit

symbol,1101,1102,1104,1108,1109,1110,1210,1213,1215,1216,...,3228,3229,3231,3259,3373,4542,4905,5355,5478,5516
datetime,,,,,,,,,,,,,,,,,,,,,
2022-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,...,-0.0,-0.0,0.0,-0.0,0.0,NaN,NaN,NaN,NaN,NaN
2022-11-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0
2022-12-01,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,-0.0,...,-0.0,0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-12-02,-0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0
2022-12-05,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,...,0.0,0.0,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-28,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,...,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,0.0
2024-03-29,0.0,0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0,0.0
2024-04-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,...,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.0


In [61]:
recent_w_profit.sum(axis =1)#.sum()

datetime
2022-11-29    0.008785
2022-11-30    0.011908
2022-12-01    0.001652
2022-12-02   -0.001208
2022-12-05    0.002191
                ...   
2024-03-28    0.003670
2024-03-29    0.000606
2024-04-01    0.004574
2024-04-02    0.002809
2024-04-03    0.001167
Length: 287, dtype: float64